In [37]:
# https://www.tensorflow.org/datasets/keras_example

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import logging, os
import pandas as pd

logging.basicConfig(level=logging.INFO)
# logging.basicConfig(level=logging.DEBUG)

BASE_PATH = "../../../../../local_data/practice/tfds/"
DATA_PATH = "../../../../../local_data/tfds/"
OUTPUT_PATH = BASE_PATH+"mnist_simple_fit/"
os.system("mkdir -p " + OUTPUT_PATH)


(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

print(f"Number of training samples: {ds_train.cardinality()}")
print(f"Number of test samples: {ds_test.cardinality()}")

# plt.figure(figsize=(10, 10))
# for i, (image, label) in enumerate(ds_train.take(9)):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(image)
#     plt.title(int(label))
#     plt.axis("off")

INFO:absl:Load dataset info from /home/tmeng12/tensorflow_datasets/mnist/3.0.1
INFO:absl:Fields info.[citation, splits, supervised_keys, module_name] from disk and from code do not match. Keeping the one from code.
INFO:absl:Reusing dataset mnist (/home/tmeng12/tensorflow_datasets/mnist/3.0.1)
INFO:absl:Constructing tf.data.Dataset mnist for split ['train', 'test'], from /home/tmeng12/tensorflow_datasets/mnist/3.0.1


Number of training samples: 60000
Number of test samples: 10000


In [38]:
c1=tfds.as_dataframe(ds_train.take(20), ds_info)
col1 = c1['label']
# col1 = pd.DataFrame(train_ds, columns=["image","train_ds"])
print(col1)


0     4
1     1
2     0
3     7
4     8
5     1
6     2
7     7
8     1
9     6
10    6
11    4
12    7
13    7
14    3
15    3
16    7
17    9
18    9
19    1
Name: label, dtype: int64


2025-07-05 16:35:51.341628: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [39]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [40]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [41]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6
469/469 [==============================] - 3s 3ms/step - loss: 0.3542 - sparse_categorical_accuracy: 0.9027 - val_loss: 0.1915 - val_sparse_categorical_accuracy: 0.9446
Epoch 2/6
469/469 [==============================] - 2s 3ms/step - loss: 0.1640 - sparse_categorical_accuracy: 0.9532 - val_loss: 0.1387 - val_sparse_categorical_accuracy: 0.9570
Epoch 3/6
469/469 [==============================] - 2s 3ms/step - loss: 0.1196 - sparse_categorical_accuracy: 0.9658 - val_loss: 0.1095 - val_sparse_categorical_accuracy: 0.9680
Epoch 4/6
469/469 [==============================] - 2s 3ms/step - loss: 0.0932 - sparse_categorical_accuracy: 0.9728 - val_loss: 0.1001 - val_sparse_categorical_accuracy: 0.9688
Epoch 5/6
469/469 [==============================] - 1s 3ms/step - loss: 0.0758 - sparse_categorical_accuracy: 0.9780 - val_loss: 0.0872 - val_sparse_categorical_accuracy: 0.9746
Epoch 6/6
469/469 [==============================] - 1s 3ms/step - loss: 0.0626 - sparse_categorical_accu

In [42]:
# pred = model.predict(test_ds)
pred = model.predict(ds_train)
print("pred\n",pred)

# col2 = pd.DataFrame(pred[0:20], columns=["p1"])
col2 = pd.DataFrame(pred[0:20], columns=["p0","p1","p2","p3","p4","p5","p6","p7","p8","p9"])
print(col2)

compare = pd.concat([col1, col2], axis=1)
print(compare)
compare.to_csv(OUTPUT_PATH + "pred.csv", index=False)


469/469 [==============================] - 1s 1ms/step
pred
 [[ -4.982441    -5.3980865   -2.0220566  ...  -1.4140185   -6.693772
   -2.1357856 ]
 [ -9.589455    -4.6805024   -4.1413274  ... -10.023688     1.2847191
   -0.16779128]
 [ -7.8583755   -3.8878503    9.492837   ...  -9.776202    -0.93961823
   -4.4986873 ]
 ...
 [ -6.2451305   -6.6068544   -2.5088427  ...  -9.003259     9.393931
   -0.5799242 ]
 [ -6.3814297    6.801446    -2.5506017  ...  -0.5635487   -2.6256201
   -5.503732  ]
 [ -8.505396    -2.1075935   -2.6739006  ... -10.7592325   -1.0060546
   -0.39955002]]
           p0         p1        p2         p3         p4        p5         p6  \
0   -4.982441  -5.398087 -2.022057  -4.477812   8.419209 -6.223377  -2.875536   
1   -9.589455  -4.680502 -4.141327   2.337778  -8.765142  8.257325  -5.988466   
2   -7.858376  -3.887850  9.492837   1.789162  -6.692420 -7.813008  -7.781231   
3  -11.777388  -0.797188 -1.375965   1.720406  -5.605047 -5.030684 -11.350565   
4   -3.762067